In [4]:
import pandas as pd
from numpy import random as npr
from modules import utils

In [5]:
SEED = 2021
npr.seed(SEED)

In [8]:
df = pd.read_csv("data/rating.csv")
df

,user_id,item_id,rating,timestamp
0,0,1104,1.0,978300760
1,0,639,0.6,978302109
2,0,853,0.6,978301968
3,0,3177,0.8,978300275
4,0,2162,1.0,978824291
...,...,...,...,...
1000204,6039,1019,0.2,956716541
1000205,6039,1022,1.0,956704887
1000206,6039,548,1.0,956704746
1000207,6039,1024,0.8,956715648


In [12]:
len(df.user_id.unique())

6040

In [13]:
len(df.item_id.unique())

3706

In [14]:
df.item_id.max()

3705

In [9]:
%%time
df = pd.read_csv("data/rating.csv")
utils.evaluate_autorec(df)

Load data finished. Number of users: 6040 Number of items: 3706
IAutoRec.
Network built
Epoch: 0000; RMSE:0.25265256458280366; MAE:0.19817208051502386
Epoch: 0003; RMSE:0.20801604988739125; MAE:0.16953075841329254
Epoch: 0006; RMSE:0.18988626751378995; MAE:0.14988018825576668
Epoch: 0009; RMSE:0.187047369750567; MAE:0.14755947785405246
Epoch: 0012; RMSE:0.184987453218864; MAE:0.1459056196496809


KeyboardInterrupt: 